In [22]:
# Dependencies 
from splinter import Browser
from bs4 import BeautifulSoup as bs
import pandas as pd
import time
import json
import datetime

In [24]:
# URL to scrape NASA 
url = 'https://mars.nasa.gov/news/'


In [25]:
# Status code 200?
response = requests.get(url)
response.status_code

200

In [26]:

html = response.text
html[0:50]

"<!DOCTYPE html>\n<html lang='en' xml:lang='en' xmln"

In [27]:
# BeautifulSoup time 
soup = BeautifulSoup(html, "lxml")

In [28]:
# Chrome Driving
executable_path = {'executable_path': "../../../chromedriver.exe"}
browser = Browser('chrome', **executable_path, headless=True)
url = 'https://mars.nasa.gov/news/'
browser.visit(url)



In [29]:
html = browser.html
soup = BeautifulSoup(html, "lxml")

In [30]:
# Article TItle
article_title = soup.find_all('div', {'class':'content_title'})[1].text.strip()
article_title

"How NASA's Mars Helicopter Will Reach the Red Planet's Surface"

In [31]:
# Article Teaser
article_teaser = soup.find_all('div', {'class':'article_teaser_body'})[1].text
article_teaser

"The Red Planet's surface has been visited by eight NASA spacecraft. The ninth will be the first that includes a roundtrip ticket in its flight plan. "

In [32]:
# Get the JPL Image
url_jpl = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url_jpl)
browser.click_link_by_id("full_image")

In [33]:
html_jpl = browser.html
soup_jpl = bs(html_jpl, "html.parser")

In [34]:
test_jpl = soup_jpl.find('img', {'class':'fancybox-image'})['src']
test_jpl

'/spaceimages/images/mediumsize/PIA23436_ip.jpg'

In [35]:
fig = 'https://www.jpl.nasa.gov' + test_jpl
fig

'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA23436_ip.jpg'

In [36]:
# Facts for Mars
marsFacts = pd.read_html('https://space-facts.com/mars/')[0]
marsFacts = marsFacts.rename(columns = {marsFacts.columns[0]:'Attribute',marsFacts.columns[1]:'Value',})
marsFacts

,Attribute,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [37]:
# The Hemisphere Images
url_hemi = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url_hemi)

In [38]:
html = browser.html
soup = bs(html, "html.parser")

In [39]:
images = soup.find_all(['a', 'img alt'], {'class':'itemLink product-item'})
images

[<a class="itemLink product-item" href="/search/map/Mars/Viking/cerberus_enhanced"><img alt="Cerberus Hemisphere Enhanced thumbnail" class="thumb" src="/cache/images/39d3266553462198bd2fbc4d18fbed17_cerberus_enhanced.tif_thumb.png"/></a>,
 <a class="itemLink product-item" href="/search/map/Mars/Viking/cerberus_enhanced"><h3>Cerberus Hemisphere Enhanced</h3></a>,
 <a class="itemLink product-item" href="/search/map/Mars/Viking/schiaparelli_enhanced"><img alt="Schiaparelli Hemisphere Enhanced thumbnail" class="thumb" src="/cache/images/08eac6e22c07fb1fe72223a79252de20_schiaparelli_enhanced.tif_thumb.png"/></a>,
 <a class="itemLink product-item" href="/search/map/Mars/Viking/schiaparelli_enhanced"><h3>Schiaparelli Hemisphere Enhanced</h3></a>,
 <a class="itemLink product-item" href="/search/map/Mars/Viking/syrtis_major_enhanced"><img alt="Syrtis Major Hemisphere Enhanced thumbnail" class="thumb" src="/cache/images/55a0a1e2796313fdeafb17c35925e8ac_syrtis_major_enhanced.tif_thumb.png"/></a>,

In [40]:
hemiLinks = []
for image in images: 
    link = 'https://astrogeology.usgs.gov' + image['href']
    hemiLinks.append(link)

In [41]:
hemiLinks = set(hemiLinks)
hemiLinks


{'https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced',
 'https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced',
 'https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced',
 'https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced'}

In [42]:
temp_link = 'https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced'
browser.visit(temp_link)
html = browser.html
soup = bs(html, "html.parser")

In [43]:
img_part = soup.find(['img'], {'class':'wide-image'})['src']
img_links = 'https://astrogeology.usgs.gov' + img_part
img_links

'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'

In [44]:
soup.find(['h2'], {'class':'title'}).text

'Cerberus Hemisphere Enhanced'

In [49]:
listHemi = []

for link in hemiLinks: 
    browser.visit(link)
    html = browser.html
    soup = bs(html, "html.parser")
    partial = soup.find(['img'], {'class':'wide-image'})['src']
    full_link = 'https://astrogeology.usgs.gov' + partial
    title = soup.find(['h2'], {'class':'title'}).text
    dict_entry = {'title':title, 'img_url':full_link}
    listHemi.append(dict_entry)

In [50]:
listHemi

[{'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'},
 {'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'}]

In [51]:
# Mars Twitter Info
url_twitter = 'https://twitter.com/marswxreport?lang=en'
browser.visit(url_twitter)

In [52]:
html_twitter = browser.html
soup_twitter = bs(html_twitter, "html.parser")

In [70]:

tweet_divs = soup_twitter.find('div',{'data-testid':'tweet'})
tweet_spans = tweet_divs.find_all('span')
tweet_spans

[<span class="css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0"><span class="css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0">Mars Weather</span></span>,
 <span class="css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0">Mars Weather</span>,
 <span class="css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0">@MarsWxReport</span>,
 <span class="css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0">·</span>,
 <span class="css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0">InSight sol 566 (2020-06-29) low -88.4ºC (-127.1ºF) high -4.2ºC (24.4ºF)
 winds from the WNW at 5.1 m/s (11.5 mph) gusting to 16.8 m/s (37.6 mph)
 pressure at 7.70 hPa</span>,
 <span class="css-901oao css-16my406 r-1qd0xha r-n6v787 r-1sf4r6n r-1n0xq6e r-bcqeeo r-d3hbe1 r-1wgg2b2 r-axxi2z r-qvutc0"><span class="css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0">13</span></span>,
 <span class="css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0">13</

In [71]:
for span in tweet_spans: 
    if "InSight sol" in span.text: 
        tweet = span.text
        break

In [72]:
tweet

'InSight sol 566 (2020-06-29) low -88.4ºC (-127.1ºF) high -4.2ºC (24.4ºF)\nwinds from the WNW at 5.1 m/s (11.5 mph) gusting to 16.8 m/s (37.6 mph)\npressure at 7.70 hPa'

In [81]:
def scrape():
    #Executable Path + Browser
    executable_path = {'executable_path': '../../../chromedriver'}
    browser = Browser('chrome', **executable_path, headless=True)
    
    
    url = 'https://mars.nasa.gov/news/'
    browser.visit(url)
    time.sleep(1)
    
    
    html = browser.html
    soup = bs(html, "html.parser")
    
    
    article_title = soup.find_all('div', {'class':'content_title'})[1].text.strip()
    
     
    article_teaser = soup.find_all('div', {'class':'article_teaser_body'})[1].text
    
    
    url_jpl = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
    browser.visit(url_jpl)
    browser.click_link_by_id("full_image")
    time.sleep(1)
    
    
    html_jpl = browser.html
    soup_jpl = bs(html_jpl, "html.parser")
    
    
    temp = soup_jpl.find('img', {'class':'fancybox-image'})['src']
    figure_url = 'https://www.jpl.nasa.gov' + temp
    
    
    url_twitter = 'https://twitter.com/marswxreport?lang=en'
    browser.visit(url_twitter)
    time.sleep(1)
    
    
    html_twitter = browser.html
    soup_twitter = bs(html_twitter, "html.parser")
    time.sleep(1)
    
    
    tweet_div = soup_twitter.find('div',{'data-testid':'tweet'})
    #tweet_spans = tweet_div.find_all('span')
    
    for span in tweet_spans: 
        if "InSight sol" in span.text: 
            tweet = span.text
            break
    
    
    mars_facts = pd.read_html('https://space-facts.com/mars/')[0]
    mars_facts = mars_facts.rename(columns = {mars_facts.columns[0]:'Attribute',mars_facts.columns[1]:'Value',})
    mars_json = json.loads(mars_facts.to_json(orient="records"))
    
    
    tempHemi = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
    browser.visit(tempHemi)
    time.sleep(1)
    
    html = browser.html
    soup = bs(html, "html.parser")
    
    images = soup.find_all(['a', 'img alt'], {'class':'itemLink product-item'})
    
    hemiLinks = []
    
    for image in images: 
        link = 'https://astrogeology.usgs.gov' + image['href']
        hemiLinks.append(link)
    hemiLinks = set(hemiLinks)
    
    
    listHemi = []
    for link in hemiLinks: 
        browser.visit(link)
        time.sleep(1)
        
        html = browser.html
        soup = bs(html, "html.parser")
        partial = soup.find(['img'], {'class':'wide-image'})['src']
        full_link = 'https://astrogeology.usgs.gov' + partial
        title = soup.find(['h2'], {'class':'title'}).text
        dict_entry = {'title':title, 'img_url':full_link}
        listHemi.append(dict_entry)
    
     
    browser.quit()
     
    marsDict = {
        "ArticleTitle": article_title,
        "ArticleTeaser": article_teaser,
        "featureImageURL": figure_url,
        "tweetWeatherText": tweet,
        "marsStats": mars_json,
        "HemisphereImages": listHemi,
        "dateScraped": datetime.datetime.now()
    }
    
    return marsDict

In [82]:
marsNews = scrape()
marsNews

{'ArticleTitle': "How NASA's Mars Helicopter Will Reach the Red Planet's Surface",
 'ArticleTeaser': "The Red Planet's surface has been visited by eight NASA spacecraft. The ninth will be the first that includes a roundtrip ticket in its flight plan. ",
 'featureImageURL': 'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA18851_ip.jpg',
 'tweetWeatherText': 'InSight sol 566 (2020-06-29) low -88.4ºC (-127.1ºF) high -4.2ºC (24.4ºF)\nwinds from the WNW at 5.1 m/s (11.5 mph) gusting to 16.8 m/s (37.6 mph)\npressure at 7.70 hPa',
 'marsStats': [{'Attribute': 'Equatorial Diameter:', 'Value': '6,792 km'},
  {'Attribute': 'Polar Diameter:', 'Value': '6,752 km'},
  {'Attribute': 'Mass:', 'Value': '6.39 × 10^23 kg (0.11 Earths)'},
  {'Attribute': 'Moons:', 'Value': '2 (Phobos & Deimos)'},
  {'Attribute': 'Orbit Distance:', 'Value': '227,943,824 km (1.38 AU)'},
  {'Attribute': 'Orbit Period:', 'Value': '687 days (1.9 years)'},
  {'Attribute': 'Surface Temperature:', 'Value': '-87 to -5 °